# 다이소 재고 검색 프로젝트

- 기능 1. 해당 상품명 검색 시 추천순(or 판매량, 리뷰많은순) 상품 5개 선택
- 기능 2. 선택된 상품들에 대해서 리뷰 데이터 수집 (별점, 내용)
- 기능 3. 입력받은 지역 근처 다이소 매장 검색 (3개)
- 기능 4. 선택된 상품과 선택한 매장에 대해 재고 검색

> 입력: 상품명, 지역  
> 출력: 상품 리스트와 리뷰, 재고

다이소몰 URL: https://www.daisomall.co.kr/ds

### 입력 받기 & 필요한 모듈 임포트

In [33]:
import time
import bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from IPython.display import clear_output 

### 상품 입력해서 상위 5개 항목의 정보 가져오는 함수


In [34]:
def get_products_info(good, browser) :
    browser.get('https://www.daisomall.co.kr/ds/dst/SCR_DST_0015?searchTerm='+good)
    time.sleep(2)
    goods = browser.find_elements(By.CLASS_NAME, "goods-unit")
    
    names = []
    prices = []
    links = []
    
    for good in goods[0:5] :
        names.append(good.find_element(By.CLASS_NAME, "tit").text.replace("BEST","").strip())
        prices.append(good.find_element(By.CLASS_NAME, "value").text)
        links.append(good.find_element(By.TAG_NAME, "a").get_attribute("href"))
        
    data = {"names" : names, "prices" : prices, "links" : links}
    df = pd.DataFrame(data=data)
    
    return df

### 상품의 상세 리뷰 가져오는 함수

In [35]:
def get_review(url, browser) :
    browser.get(url)
    time.sleep(2)
    
    response = browser.page_source
    
    review_page = bs4.BeautifulSoup(response)
    
    one_page_all_reviews = review_page.find_all('div',{'class':'cont'})
    one_page_all_scores = review_page.find_all('span',{'class':'score'})
    
    if not one_page_all_reviews :
        return pd.DataFrame()
    
    else:
        score_list = [score.get_text(strip = True) for score in one_page_all_scores]
        review_list = [review.get_text(strip = True) for review in one_page_all_reviews]
    
    df = pd.DataFrame({'text': review_list, 'score': score_list})
    
    return df


### 해당 지역의 다이소 매장 찾는 함수

In [36]:
def get_store(region, browser):
    search_url = 'https://www.daisomall.co.kr/ms/msg/SCR_MSG_0019'
    browser.get(search_url)

    time.sleep(3) 
    
    btn = browser.find_element(by=By.XPATH, value='//*[@id="btn-close-nday"]')
    btn.click()
    time.sleep(1)
    
    # 매장 검색 박스
    search_box = browser.find_elements(by=By.CLASS_NAME, value='el-input__inner')[1] # index 0: product search box
    search_box.clear()
    search_box.send_keys(region)
    search_box.send_keys(Keys.ENTER)

    time.sleep(3)
    
    region_boxies = browser.find_elements(by=By.CLASS_NAME, value='store-map')

    store_list = []
    
    for i, region_box in enumerate(region_boxies):
        store = region_box.find_element(by=By.CLASS_NAME, value='tit-h5').text
        location = region_box.find_element(by=By.XPATH, value='//*[@id="pane-2"]/div/div[2]/div[{}]/div[2]/div[2]'.format(i+1)).text
        info = region_box.find_element(by=By.CLASS_NAME, value='info-group')
        opening = info.find_elements(by=By.TAG_NAME, value='span')[0].text.replace('영업 시간 ', '')
        tel = info.find_elements(by=By.TAG_NAME, value='span')[1].text.replace('매장 전화 ', '')
    
        store_list.append({'store':store.split()[0], 'location':location, 'opening':opening, 'tel':tel})
        
        store_df = pd.DataFrame(store_list)
        
    return store_df

### 특정 상품의 특정 매장에서의 재고 검색

In [37]:
def get_stock(product_no, store, browser):
    url = 'https://www.daisomall.co.kr/ms/msg/SCR_MSG_0019'
    browser.get(url)
    time.sleep(2)
    
    # step 1. 매장 상품 찾기 클릭
    btn = browser.find_element(by=By.XPATH, value='//*[@id="tab-3"]/button')
    btn.click()
    time.sleep(1)
    
    # step 2. 특정 상품 검색 및 선택
    search_box = browser.find_element(by=By.XPATH, value='//*[@id="pane-3"]/div/div[2]/div[1]/form/div/div/div/div/input')
    search_box.clear()
    search_box.send_keys(product_no)
    search_box.send_keys(Keys.ENTER)
    time.sleep(1)
    
    search_btn = browser.find_element(by=By.XPATH, value='//*[@id="__layout"]/section/div[1]/div/div[3]/div/div[2]/div/div[1]/button')
    search_btn.click()
    time.sleep(1)
    
    check_btn = browser.find_element(by=By.XPATH, value='//*[@id="__layout"]/section/div[1]/div/div[3]/div/div[2]/div/div[2]/div[1]/div[1]/label')
    check_btn.click()
    time.sleep(1)
    
    choice_btn = browser.find_element(by=By.XPATH, value='//*[@id="__layout"]/section/div[1]/div/div[3]/div/div[3]/div/button')
    choice_btn.click()
    time.sleep(1)
    
    # step 3. 해당 매장 검색 및 선택
    search_box = browser.find_element(by=By.XPATH, value='//*[@id="pane-3"]/div/div[2]/div[2]/form/div/div/div/div/div[1]/input')
    search_box.clear()
    search_box.send_keys(store)
    search_box.send_keys(Keys.ENTER)
    time.sleep(1)
    
    check_btn = browser.find_element(by=By.CLASS_NAME, value='el-radio__input')
    check_btn.click()
    time.sleep(1)
    
    choice_btn = browser.find_element(by=By.XPATH, value='//*[@id="__layout"]/section/div[1]/div/div[4]/div/div[3]/div/button')
    choice_btn.click()
    time.sleep(1)
    
    # step 4. 재고 정보 가져오기
    stock = browser.find_element(by=By.XPATH, value='//*[@id="pane-3"]/div/div[2]/div[3]/div/div[1]/div[4]/em[2]').text
    zone = browser.find_element(by=By.XPATH, value='//*[@id="pane-3"]/div/div[2]/div[3]/div/div[1]/div[4]/em[1]').text
    
    # step 5. 재고 결과 처리
    if '품절' in stock:
        stock = '품절'
    elif '단종' in stock:
        stock = '단종'
    elif stock == '-':
        stock = '-'
    else:
        stock = ' '.join(stock.split()[1:3])
        
    zone = zone.split()[-1]
        
    return stock, zone

---

### 메인

In [38]:
# 브라우저 옵션 설정 및 브라우저 실행
options = webdriver.EdgeOptions()
options.add_argument('--no-sandbox')                    
options.add_argument('--disable-dev-shm-usage')    

driver = webdriver.Edge(options = options)
driver.maximize_window()

# DataFrame 설정
pd.set_option('display.max_colwidth',500)

In [40]:
# 상품 정보
product = input('상품명을 입력해주세요\n')
df=get_products_info(product, driver)
display(df)

상품명을 입력해주세요
젓가락


,names,prices,links
0,대나무 개별 포장 젓가락 50매입,"1,000",https://www.daisomall.co.kr/pd/pdr/SCR_PDR_0001?pdNo=1042210
1,실리콘 수저 젓가락 받침 세트,"1,000",https://www.daisomall.co.kr/pd/pdr/SCR_PDR_0001?pdNo=1039701
2,캠핑용 조립식 젓가락 파우치,"2,000",https://www.daisomall.co.kr/pd/pdr/SCR_PDR_0001?pdNo=1040290
3,스텐 진공 젓가락 5세트,"2,000",https://www.daisomall.co.kr/pd/pdr/SCR_PDR_0001?pdNo=1003940
4,일회용 젓가락 20매입,500,https://www.daisomall.co.kr/pd/pdr/SCR_PDR_0001?pdNo=1016872


In [41]:
# 리뷰 정보
review_ls = []
for url in df["links"]:
    review_ls.append(get_review(url,driver))

In [42]:
# 리뷰 출력
while True :
    answer = input("리뷰를 보고 싶은 상품을 숫자로 선택해주세요(0~4) 보고 싶은 상품이 없을 경우 \"N\"을 입력해주세요")
    clear_output(wait=True)
    if answer == "N" :
        break
    else :
        if review_ls[int(answer)].empty:
            print("리뷰가 없습니다.")
        else :
            display(review_ls[int(answer)])

,text,score
0,재구매집들이할때 필수품!,5점
1,우리가 알고 있는 나무 젓가락입니다. 기본에 충실!,5점
2,좋아요~~ 생각보다 괜찮아요 잘쓸게요,5점
3,잘 받았어요.감사합니다..잘 받았어요감사합니다,5점
4,저렴하고 양이 많아 좋아요 나무젓가락도 유통기한이 있대서 적당한 양으로 구입,5점
5,일반 젓가락인데 개별포장이 되어있어서 좋아요,5점
6,아직 사용전이지만 싼 가격에 샀습니다 쉽게 안 뿌서지면 좋겠네용 ㅎㅎ,5점
7,개별포장이라서 좋음.,5점
8,오 생각보다 크고 좋아요 저렴해서 또 구매할듯요!ㅎㅎ 추천합니다,5점
9,가성비 짱입니다깔끔하게 갈라집니다,5점


리뷰를 보고 싶은 상품을 숫자로 선택해주세요(0~4) 보고 싶은 상품이 없을 경우 "N"을 입력해주세요N


In [43]:
# 매장 정보
region = input("매장을 찾고 싶은 곳의 지역을 입력해주세요.\n")

store = get_store(region, driver)
display(store)

매장을 찾고 싶은 곳의 지역을 입력해주세요.
대구


,store,location,opening,tel
0,대구감삼점,대구광역시 달서구 달구벌대로 1657 (감삼동),10:00 ~ 22:00,1522-4400
1,대구내당점,대구광역시 서구 평리로 250(내당동),10:00 ~ 22:00,053-566-6016
2,대구논공점,대구광역시 달성군 논공읍 용호로 26,10:00 ~ 22:00,053-615-0177
3,대구대현점,대구광역시 북구 신암로 111 (대현동),10:00 ~ 22:00,1522-4400
4,대구복현점,대구광역시 북구 동북로 290 (복현동),10:00 ~ 22:00,1522-4400
5,대구본리점,대구광역시 달서구 와룡로 130 (감삼동),09:30 ~ 22:00,1522-4400
6,대구서부점,대구광역시 달서구 구마로 265 (성당동),10:00 ~ 22:00,1522-4400
7,대구서재점,대구광역시 달성군 다사읍 서재로 110,10:00 ~ 22:00,1522-4400
8,대구신당점,대구광역시 달서구 계대동문로 23-1 (신당동),10:00 ~ 22:00,1522-4400
9,대구옥포점,대구광역시 달성군 옥포읍 돌미로 10,10:00 ~ 22:00,1522-4400


In [46]:
# 재고 검색
product_num = int(input("아까 본 상품 중 맘에 드는 상품을 숫자로 골라주세요.(0~4)\n"))
input_store = input("해당 상품을 찾고 싶은 매장명을 입력해주세요.\n")
stock, location = get_stock(df["links"][product_num][-7:], input_store, driver)

product_name = df.iloc[product_num].names
product_prices = df.iloc[product_num].prices

print('\n\n')
print(f'>>> 다이소 {input_store}의 {product_name}({product_prices}원) 재고는 {stock}입니다. (위치: {location})')

아까 본 상품 중 맘에 드는 상품을 숫자로 골라주세요.(0~4)
0
해당 상품을 찾고 싶은 매장명을 입력해주세요.
대구신당점



>>> 다이소 대구신당점의 대나무 개별 포장 젓가락 50매입(\1,000) 재고는 14개 이하입니다. (위치: 19)


In [47]:
# 드라이버 종료
driver.quit()